In [4]:
import tensorflow as tf
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from tensorflow.contrib.layers import fully_connected

__author__ = "ebo"

mnist = tf.keras.datasets.mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()


# X转为784列的矩阵
X_train = X_train.reshape((-1, 784))
X_test = X_test.reshape((-1, 784))

''' 这里是个坑，如果不进行数据归一化，得到的准确率非常低'''
X_train = X_train / 255.0
X_test = X_test / 255.0


# y值转为oneHot编码
one_hot_encoder = OneHotEncoder(sparse=False)
y_train_one_hot = one_hot_encoder.fit_transform(y_train.reshape(len(y_train), 1))
y_test_one_hot = one_hot_encoder.fit_transform(y_test.reshape(len(y_test), 1))


n_inputs = 28 * 28
n_hidden1 = 300
n_hidden2 = 100
n_output = 10

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="x")
y = tf.placeholder(tf.int64, shape=(None, n_output), name='y')


# 定义隐藏层的计算逻辑
def neuron_layer(X, n_neuron, name, activation=None):
    
    with tf.name_scope(name):
        
        n_inputs = int(X.shape[1])
        stddev = 2 / np.sqrt(n_inputs)
        init = tf.truncated_normal((n_inputs, n_neuron), stddev=stddev)
        
        w = tf.Variable(init, name="weight")
        b = tf.Variable(tf.zeros([n_neuron]), name="biases")
        
        z = tf.matmul(X, w) + b
        
        if activation == "relu":
            return tf.nn.relu(z)
        else:
            return z

# 创建神经网络隐藏层
with tf.name_scope("dnn"):
    hidden1 = neuron_layer(X, n_hidden1, "hidden1", activation="relu")
    hidden2 = neuron_layer(hidden1, n_hidden2, "hidden2", activation="relu")
    logits = neuron_layer(hidden2, n_output, "output")
    
# 创建损失函数
with tf.name_scope("loss"):
    xentropy = tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name='loss')
    
learning_rate = 0.01

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    train_op = optimizer.minimize(loss)
    

with tf.name_scope("eval"):
    
#     correct = tf.nn.in_top_kv(logits, y, 1)
#     accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    

init = tf.global_variables_initializer()
saver = tf.train.Saver()

n_epoch = 400
batch_size = 50

with tf.Session() as sess:
    
    init.run()
    
    for epoch in range(n_epoch):
        for i in range(len(X_train) // batch_size):
            batch_x = X_train[i*batch_size:min(i*batch_size+batch_size, len(X_train))]
            batch_y = y_train_one_hot[i*batch_size:min(i*batch_size+batch_size, len(X_train))]
            sess.run(train_op, feed_dict={X: batch_x, y: batch_y})
            acc_train = accuracy.eval(feed_dict={X: batch_x, y: batch_y})
            acc_test = accuracy.eval(feed_dict={X: X_test, y: y_test_one_hot})
            
#             print(i, "train acc", acc_train, "test acc ", acc_test)
            
            if i % 50 == 0:
                print("epoch", epoch, "iteration",i, "train acc", acc_train, "test acc ", acc_test)
                
                
            if i % 100 == 0:
                saver.save(sess, "./model/handwriting_dnn.ckpt")
                

        



' 这里是个坑，如果不进行数据归一化，得到的准确率非常低'

epoch 0 iteration 0 train acc 0.12 test acc  0.12


'./model/handwriting_dnn.ckpt'

epoch 0 iteration 50 train acc 0.64 test acc  0.5926
epoch 0 iteration 100 train acc 0.78 test acc  0.7436


'./model/handwriting_dnn.ckpt'

...

epoch 0 iteration 650 train acc 0.82 test acc  0.8934
epoch 0 iteration 700 train acc 0.96 test acc  0.8985


'./model/handwriting_dnn.ckpt'

epoch 0 iteration 750 train acc 0.84 test acc  0.9017
epoch 0 iteration 800 train acc 0.94 test acc  0.9011


'./model/handwriting_dnn.ckpt'

KeyboardInterrupt: 